In [1]:
# Imports
import numpy as np

import pandas as pd

import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as dhtml
from dash.dependencies import Input, Output

from sklearn import linear_model

from scipy import signal


## CRISP-DM
![CRISP-DM](../reports/figures/CRISP_DM.png)

# Modelling- Spread
- We want to focus on modelling how the virus spreads

In [2]:
# Utility plotting function
def quick_plot(x_data, y_data, type="log", xslider=False):
    """ Basic plot for rapid evaluation of time series data

    Parameters:
    ----------
    x_data: array
        array of numbers or datetime objects
    y_data: pandas dataframe
        matrix to plot, each column is plotted as a trace
        the column name would be used as legend entry for the trace 
    type: str
        y-axis scale, 'log' or 'linear'
    xslider: bool
        x-axis slider, True or False

    Returns:
    -------

    """
    # Create figure
    fig= go.Figure()

    # Column list
    column_list= y_data.columns
    # Loop through each column
    for column in column_list:
        # Add a trace
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=y_data[column],
                mode="lines",
                opacity=0.8,
                line_width=2,
                marker_size=5,
                name= column
            )
        )

    # Set figure layout
    fig.update_layout(
        width=900,
        height=600,
        xaxis_title="Time",
        xaxis={
            "tickangle": -75,
            "nticks": 20,
            "tickfont": dict(size=14, color="#7f7f7f")
        },
        yaxis_title="Quick Plot",
        yaxis={
            "type": type
        }
    )

    # Introduce range slider on x-axis
    if(xslider):
        fig.update_layout(xaxis_rangeslider_visible=True)

    # Display figure
    fig.show()


In [3]:
# Import compact dataset
df_base= pd.read_csv("../data/processed/COVID_flat_small.csv", sep=";")
df_base.tail()

,date,Spain,Nigeria,Germany,Afghanistan,Italy
201,2020-08-10,322980,46867,218508,37162,250825
202,2020-08-11,326612,47290,219540,37269,251237
203,2020-08-12,329784,47743,220859,37345,251713
204,2020-08-13,337334,48116,222281,37424,252235
205,2020-08-14,342813,48445,223791,37431,252809


In [4]:
# Try quick plot
quick_plot(df_base.iloc[:, 0], df_base.iloc[:, 1:], type="linear", xslider=True)

In [5]:
# Align time series to coincide at some pre-defined threshold
# This allows us to compare behaviour

threshold= 1000


In [6]:
country_list= df_base.columns.drop("date")

In [7]:
# Temporary list
temp_list=[]

for pos,country in enumerate(country_list):
    # Slice off parts of the dataframe that's above the specified threshold and append to list
    temp_list.append(np.array(df_base[country][df_base[country]>threshold]))

In [8]:
# Push list into DataFrame
df_thres= pd.DataFrame(temp_list)
df_thres.tail()

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
0,1073,1695,2277,2277,5232,6391,7798,9942,11748,13910,...,342813.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1095,1182,1273,1337,1532,1728,1932,2170,2388,2558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1040,1176,1457,1908,2078,3675,4585,5795,7272,9257,...,222281.0,223791.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1128,1694,2036,2502,3089,3858,4636,5883,7375,9172,...,248803.0,249204.0,249756.0,250103.0,250566.0,250825.0,251237.0,251713.0,252235.0,252809.0


In [9]:
# Set country names as index
df_thres= df_thres.set_index(country_list)
df_thres.head()

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
Spain,1073,1695,2277,2277,5232,6391,7798,9942,11748,13910,...,342813.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nigeria,1095,1182,1273,1337,1532,1728,1932,2170,2388,2558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Germany,1040,1176,1457,1908,2078,3675,4585,5795,7272,9257,...,222281.0,223791.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Italy,1128,1694,2036,2502,3089,3858,4636,5883,7375,9172,...,248803.0,249204.0,249756.0,250103.0,250566.0,250825.0,251237.0,251713.0,252235.0,252809.0


In [10]:
# Transpose
df_thres= df_thres.T
df_thres

,Spain,Nigeria,Germany,Afghanistan,Italy
0,1073.0,1095.0,1040.0,1026.0,1128.0
1,1695.0,1182.0,1176.0,1092.0,1694.0
2,2277.0,1273.0,1457.0,1176.0,2036.0
3,2277.0,1337.0,1908.0,1279.0,2502.0
4,5232.0,1532.0,2078.0,1351.0,3089.0
...,...,...,...,...,...
163,NaN,NaN,NaN,NaN,250825.0
164,NaN,NaN,NaN,NaN,251237.0
165,NaN,NaN,NaN,NaN,251713.0
166,NaN,NaN,NaN,NaN,252235.0


In [11]:
# x_data for plot
x_data= np.arange(df_thres.shape[0])

# Quick Plot
quick_plot(x_data, df_thres)

**Exponential Function**  
$N(t, T)= N_0 * 2^{t/T}$  
$N_{0}$- base value  
t- time  
T- base time (i.e. $N(t,2)$ is the doubling rate for every 2 days)  

In [12]:
def doubled_series(N_0, t, T):
    """ Calculate the doubled time series for a specified doubling rate

    Parameters:
    ----------
    N_0: double
        initial value
    t: array Nx1
        time
    T: int
        base time

    Returns:
    -------
    doubling_rate: array Nx1
    """
    return N_0 * np.power(2, (t/T))

In [13]:
# Calculate the doubled series for different doubling rates
d_rates= [10,12,15]

# Will hold output series
df_series= {}

for rate in d_rates:
    df_series["doubling every {0} days".format(rate)]= doubled_series(threshold, x_data, rate)

In [14]:
# Concatenate doubled series with synchronized timeline series
df_sync_with_slopes= pd.concat([pd.DataFrame(df_series), df_thres], axis=1)

In [15]:
# Plot Data with doubled series (which are essentially normalized slopes)
quick_plot(x_data, df_sync_with_slopes)

In [16]:
# Save dataset
df_sync_with_slopes.to_csv("../data/processed/COVID_small_sync_timelines.csv", sep=";")

## Understanding Linear Regression

### Nigeria

In [17]:
# Create Linear Regression Model
reg= linear_model.LinearRegression(fit_intercept= False)    
#fit_intercept= False sets the y-axis intercept to zero.

In [18]:
# Setup fitting data
y= np.array(df_base["Nigeria"])

In [19]:
y.shape

(206,)

In [20]:
# First non-zero index
not_0_idx= np.where(y>0)
not_0_idx

(array([ 37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,
         50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
         63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
         76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
         89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
        102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
        115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
        167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
        180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
        193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205]),)

In [21]:
# Truncate data to start from first non-zero index
y= y[not_0_idx[0][0]:]

In [22]:
# scikit-learn takes X matrix with rows as features and columns as samples
nig_row= np.arange(y.shape[0])
X= nig_row.reshape(-1, 1)
#np.reshape(-1,1) infers the row number from the array upon which it is applied

#### Linear Regression Fit (Nigeria)

In [23]:
reg.fit(X,y)

LinearRegression(fit_intercept=False)

In [24]:
# Inspect Solution
X_hat= X
y_hat= reg.predict(X_hat)

In [25]:
LR_inspect= pd.DataFrame([df_base["date"], df_base["Nigeria"]])
LR_inspect.head()

,0,1,2,3,4,5,6,7,8,9,...,196,197,198,199,200,201,202,203,204,205
date,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14
Nigeria,0,0,0,0,0,0,0,0,0,0,...,44890,45244,45687,46140,46577,46867,47290,47743,48116,48445


In [26]:
LR_inspect= LR_inspect.T
LR_inspect.tail()

,date,Nigeria
201,2020-08-10,46867
202,2020-08-11,47290
203,2020-08-12,47743
204,2020-08-13,48116
205,2020-08-14,48445


In [27]:
LR_inspect.set_index= "date"
# Truncate data to start from first non-zero index
LR_inspect= LR_inspect[not_0_idx[0][0]:]

In [28]:
LR_inspect["prediction"]= y_hat

In [29]:
LR_inspect.tail()

,date,Nigeria,prediction
201,2020-08-10,46867,33859.090889
202,2020-08-11,47290,34065.548761
203,2020-08-12,47743,34272.006632
204,2020-08-13,48116,34478.464503
205,2020-08-14,48445,34684.922375


In [30]:
# Quick Plot
quick_plot(LR_inspect["date"], LR_inspect.iloc[:,1:], type='linear', xslider=True)

### Data Transformation to Log Domain
Here, we transform out data into the Logarithimic domain before applying linear regression.  
However, it's imperative to remember that the prediction outcome must be transformed back to 
linear domain for comparison with original data.

In [31]:
reg= linear_model.LinearRegression()

In [32]:
# Transform data to log domain
y= np.log(np.array(df_base["Nigeria"]))
# Slice out zero-values
y= y[not_0_idx[0][0]:]

In [33]:
# X data
nig_row= np.arange(y.shape[0])
X= nig_row.reshape(-1, 1)

# Linear Regression
reg.fit(X,y)

# Get estimated values
y_hat= reg.predict(X)

# Convert back to linear domain
y_hat= np.exp(y_hat)

# Replace predicted values in LR_inspect
LR_inspect["prediction"]= y_hat

# Make quick plot
quick_plot(LR_inspect["date"], LR_inspect.iloc[:,1:], type='log', xslider=True)

### Piecewise Linear Regression- Doubling Rate

In [34]:
def get_rate_via_regression(in_array):
    """ Approximate the doubling time using linear regression.

    3 datapoints are used to approximate the number of days 
    it takes for the number of infected people to double at each point.

    Parameters:
    ----------
    in_array: list/numpy array
        input data

    Returns:
    -------
    doubling_time: double
    """
    
    # Assert output vector is 3 datapoints long
    assert len(in_array)==3

    y= np.array(in_array)
    # Calculate slope using central difference
    # Remeber: scikit-learn takes X matrix with rows as features and columns as samples
    X= np.arange(-1,2).reshape(-1,1)

    # Fit data
    reg.fit(X,y)
    intercept= reg.intercept_
    slope= reg.coef_

    return intercept/slope


In [35]:
def doubling_time_closed(in_array):
    """ Calculate the doubling time in closed form
    """
    y= np.array(in_array)

    return len(y)*np.log(2)/np.log(y[-1]/y[0])

#### Doubling Time for Nigeria

In [36]:
# Setup fitting data
fitt_data= df_base["Nigeria"]

#### **rolling()** in Pandas
This function moves a window across a time series

In [37]:
# Doubling time (apprx)
fitt_db_time= fitt_data.rolling(
    window=3,
    min_periods=3
).apply(get_rate_via_regression, raw=False)

In [38]:
# Doubling time (closed form)
fitt_db_closed= fitt_data.rolling(
    window=3,
    min_periods=3
).apply(doubling_time_closed)

In [39]:
# Quick plot
quick_plot(df_base["date"], pd.DataFrame({"apprx": fitt_db_time, "closed": fitt_db_closed}), type='linear')

### Cleaning up the input data
- We want to filter the data in such a way that we do not distort the trend.

#### Savitzky-Golay Filter

In [40]:
country_filter_list= []
# Filter input data
for ctry in country_list:
    df_base[ctry + '-filter']= signal.savgol_filter(
        df_base[ctry],
        3, # filtering window
        1, # order of polynomial with which to fit
    )
    # Add to filter list
    country_filter_list.append(ctry + '-filter')


In [41]:
df_base.columns

Index(['date', 'Spain', 'Nigeria', 'Germany', 'Afghanistan', 'Italy',
       'Spain-filter', 'Nigeria-filter', 'Germany-filter',
       'Afghanistan-filter', 'Italy-filter'],
      dtype='object')

#### Run doubling time estimation on cleaned data

In [42]:
# No of days over which to estimate
days_over= 3

for ctry in country_filter_list:
    df_base[ctry + '-dt']= df_base[ctry].rolling(
        window=days_over,
        min_periods=days_over
    ).apply(get_rate_via_regression, raw=False)

In [43]:
df_base.columns

Index(['date', 'Spain', 'Nigeria', 'Germany', 'Afghanistan', 'Italy',
       'Spain-filter', 'Nigeria-filter', 'Germany-filter',
       'Afghanistan-filter', 'Italy-filter', 'Spain-filter-dt',
       'Nigeria-filter-dt', 'Germany-filter-dt', 'Afghanistan-filter-dt',
       'Italy-filter-dt'],
      dtype='object')

#### Plot doubling rates

In [44]:
# Starting position
start_pos= 100

quick_plot(
    df_base.iloc[start_pos:,0], df_base.iloc[start_pos:, [12, 13,14,15,16]],
    type='linear', xslider=True
)

IndexError: positional indexers are out-of-bounds